In [118]:
from pyscf import gto, scf, mcscf, fci
from pyscf.fci import cistring
import openfermion as of
import cirq
from openfermion.circuits import trotter
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt
import scipy
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
import cirq
import openfermion
from openfermion import InteractionOperator
from openfermion.circuits import trotter
from openfermion.transforms import get_interaction_operator
from cirq.testing import random_circuit


 done


  Preparing metadata (pyproject.toml) ... -

 done


  Using cached cirq_core-1.4.1-py3-none-any.whl.metadata (1.8 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl.metadata (4.6 kB)


  Using cached PubChemPy-1.0.4-py3-none-any.whl


...

 \

 |

 /

 done
  Created wheel for openfermion: filename=openfermion-1.7.0.dev0-py3-none-any.whl size=1209626 sha256=c465373d77b9e9b4e01d7044a26013e6b7ea3d2d367cc5f664078a49e3d96e6a
  Stored in directory: /tmpfs/tmp/pip-ephem-wheel-cache-1i0mk48_/wheels/7a/93/a6/5746203c4afd3b6e8d40a4edb587d608c51290286c5680f422


Successfully built openfermion


In [119]:
# Initialize a random initial state.
random_seed = 8317
initial_state = openfermion.haar_random_vector(
    2 ** n_qubits, random_seed).astype(np.complex64)

# Numerically compute the correct circuit output.

hamiltonian_sparse = openfermion.get_sparse_operator(molecular_hamiltonian)
exact_state = scipy.sparse.linalg.expm_multiply(
    -1j * time * hamiltonian_sparse, initial_state)

# Trotter step parameters.
n_steps = 3

# Compile the low rank Trotter step using OpenFermion.
qubits = cirq.LineQubit.range(n_qubits)
circuit = cirq.Circuit(
    trotter.simulate_trotter(
            qubits, molecular_hamiltonian,
            time=time, n_steps=n_steps,
            algorithm=trotter.LOW_RANK),
    strategy=cirq.InsertStrategy.EARLIEST)

# Use Cirq simulator to apply circuit.
simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=qubits, initial_state=initial_state)
simulated_state = result.final_state_vector

# Print final fidelity.
fidelity = abs(np.dot(simulated_state, np.conjugate(exact_state))) ** 2
print('Fidelity with exact result is {}.\n'.format(fidelity))

# Print circuit.
cirq.drop_negligible_operations(circuit)
print(circuit.to_text_diagram(transpose=True))

Fidelity with exact result is 0.9997277906419498.

0             1                     2                3
│             │                     │                │
Rz(π)         Rz(π)                 Rz(π)            Rz(π)
│             │                     │                │
│             PhISwap(0.25)─────────PhISwap(0.25)^-1 │
│             │                     │                │
PhISwap(0.25)─PhISwap(0.25)^0.081   Z^0              │
│             │                     │                │
Rz(0.337π)    Z^0                   PhISwap(0.25)────PhISwap(0.25)^-0.081
│             │                     │                │
Rz(0)         PhISwap(0.25)─────────PhISwap(0.25)^-1 Z^0
│             │                     │                │
│             Rz(0.337π)            Z^0              Rz(0.169π)
│             │                     │                │
│             Rz(0)                 Rz(0.169π)       Rz(0)
│             │                     │                │
│             │                

Double-factorized one-norm $\lambda(H_{DF})$ as a function of the squared free-fermions and eigenvalues up to rank $L = O(N)$

$$
\lambda(H_{DF}) = \sum_{ij} |h_{ij}^{(1)}| + \sum_{\ell=0}^{L-1} |\lambda_\ell| \left(\sum_{pq} |\left[g_{\ell}\right]_{pq}|\right)^2 .
$$

In [120]:

def create_controlled_trotter_circuit(hamiltonian: InteractionOperator, num_qubits: int, truncation_threshold: float = 1e-8, final_rank: int = None, spin_basis: bool = True):
    # Create the qubits for the system
    system_qubits = [cirq.NamedQubit(f'q{i}') for i in range(num_qubits)]
    
    # Add the control qubit
    control_qubit = cirq.NamedQubit('control')
    
    # Create the ControlledAsymmetricLowRankTrotterStep instance
    trotter_step = trotter.algorithms.low_rank.ControlledAsymmetricLowRankTrotterStep(
        hamiltonian=hamiltonian,
        truncation_threshold=truncation_threshold,
        final_rank=final_rank,
        spin_basis=spin_basis
    )
    
    return trotter_step


In [131]:
# Set Hamiltonian parameters for LiH simulation in active space.
diatomic_bond_length = 1.45
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1
active_space_start = 1
active_space_stop = 3

# Generate and populate instance of MolecularData.
molecule = of.MolecularData(geometry, basis, multiplicity, description="1.45")
molecule.load()

# Get the Hamiltonian in an active space.
hamiltonian = molecule.get_molecular_hamiltonian(
    occupied_indices=range(active_space_start),
    active_indices=range(active_space_start, active_space_stop))
# Create the controlled trotter circuit
controlled_trotter_circuit = create_controlled_trotter_circuit(hamiltonian,final_rank=2, num_qubits=n_qubits)

circ=cirq.Circuit()
n_qubits = of.count_qubits(molecular_hamiltonian)


qubits = cirq.LineQubit.range(n_qubits+1)
# H gate


In [132]:
from cirq.testing import random_circuit

# Parameters
num_qubits = 4  # Number of qubits
n_moments = 10  # Number of moments (depth) of the random circuit
op_density = 0.7  # Operation density (probability of a gate appearing in a moment)

# Generate a random circuit
random_circ = random_circuit(qubits[1:], n_moments, op_density)


# Print the combined circuit
print("Random Circuit:")
print(random_circ)


Random Circuit:
      ┌──┐   ┌──┐           ┌──────┐
1: ────────────@────S────────X─────────────Y───X───@───────────
               │                               │   │
2: ─────X─────Y┼──────────────iSwap────Z───S───@───┼───────Z───
        │      │              │                    │
3: ────H┼─────T┼────iSwap────X┼────────────@───────X───@───@───
        │      │    │         │            │           │   │
4: ─────@──────@────iSwap─────iSwap────────X───────X───@───@───
      └──┘   └──┘           └──────┘


In [133]:
circ+=random_circ
circ+=cirq.H.on(qubits[0])

# CU(t, n_steps)
t1, t2 =1., 2.
circ+=controlled_trotter_circuit.trotter_step(
    qubits=qubits[1:],
    time=t1,
    control_qubit=qubits[0])


circ+=controlled_trotter_circuit.trotter_step(
    qubits=qubits[1:],
    time=t2,
    control_qubit=qubits[0])

imag=True
if imag:
    circ+=cirq.S(qubits[0])**-1
    

# H gate
circ+=cirq.H.on(qubits[0])


circ+=cirq.measure(qubits[0], key='anc.')

In [134]:
# Print circuit.
cirq.drop_negligible_operations(circ)
gateset = cirq.CZTargetGateset(allow_partial_czs=False)
cz_circuit = cirq.optimize_for_target_gateset(circ, gateset=gateset)

print("Circuit compile time (to CZ gate set)")
depth = len(cirq.Circuit(cz_circuit.all_operations()))
print("Circuit Depth: "+str(depth))
print("-------------------------------------")
"""
    Get single and two qubit gates
"""
single_qubit_gates = 0
cz_gates = 0

for ops in cz_circuit.all_operations():
    
    if isinstance(ops.gate, cirq.PhasedXZGate):
        single_qubit_gates += 1
    
    else:
        cz_gates += 1
        
print("Single qubit gate count: " + str(single_qubit_gates))
print("CZ gate count: " + str(cz_gates))
print(" ")

Circuit compile time (to CZ gate set)
Circuit Depth: 568
-------------------------------------
Single qubit gate count: 805
CZ gate count: 334
 


In [135]:
print(circ.to_text_diagram(transpose=True))

  0          1             2                    3                4
  │          │             │                    │                │
┌╴│          │             │                    │                │                   ╶┐
│ H          │             │                    H                │                    │
│ │          │             X────────────────────┼────────────────@                    │
└╴│          │             │                    │                │                   ╶┘
  │          │             │                    │                │
┌╴│          │             │                    │                │                   ╶┐
│ │          │             Y                    T                │                    │
│ │          @─────────────┼────────────────────┼────────────────@                    │
└╴│          │             │                    │                │                   ╶┘
  │          │             │                    │                │
  │          S             │